# Journey North Larva Sightings

In [1]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-19-2022

# geoJSONifier

In [2]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

# Load file

In [3]:
JNorth1996_2020_df = pd.read_csv("UWMadisonArb_JNorth_Project_MonarchsAndMilkweed_1996-2020.csv")
JNorth1996_2020_df.head()

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


id sighting_date                species  number  latitude  \
0  850841703    1996-12-16  Monarch Adult Sighted       1    27.594   
1  851824801    1996-12-06  Monarch Adult Sighted       1    30.420   
2  854409931    1996-09-21  Monarch Adult Sighted       1    40.817   
3  854557206    1996-09-21  Monarch Adult Sighted       1    40.817   
4  855061862    1997-01-01  Monarch Adult Sighted       1    34.244   

   longitude  flag_location  \
0    -81.362              0   
1    -87.224              0   
2    -96.710              0   
3    -96.710              0   
4    -77.946              0   

                                            comments  \
0  On the ballfield a solitary monarch butterfly ...   
1  Our 5th grade Discovery students who went alon...   
2  We saw several hundred monarchs in our school ...   
3  Over a hundered monarchs in our school butterf...   
4  I Thought I Saw It, But I Was Suprised To See ...   

                           school  grade    image_url  customerid  number_old  \
0  Walker memorial junior academy   10.0  -999999.999         135 -999999.999   
1      Hambrick elementary school    5.0  -999999.999         150 -999999.999   
2                     -999999.999    5.0  -999999.999     9701054 -999999.999   
3     Fredstrom elementary school    6.0  -999999.999         175 -999999.999   
4                         Gregory    3.0  -999999.999         181 -999999.999   

   flag_number        species_old  flag_species  
0            1  Monarch Butterfly             1  
1            1  Monarch Butterfly             1  
2            1  Monarch Butterfly             1  
3            1  Monarch Butterfly             1  
4            1  Monarch Butterfly             1

# Data Exploratory

In [4]:
JNorth1996_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227433 entries, 0 to 227432
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             227433 non-null  int64  
 1   sighting_date  227433 non-null  object 
 2   species        227433 non-null  object 
 3   number         227433 non-null  int64  
 4   latitude       227433 non-null  float64
 5   longitude      227433 non-null  float64
 6   flag_location  227433 non-null  int64  
 7   comments       227433 non-null  object 
 8   school         227433 non-null  object 
 9   grade          227433 non-null  float64
 10  image_url      227433 non-null  object 
 11  customerid     227433 non-null  int64  
 12  number_old     227433 non-null  float64
 13  flag_number    227433 non-null  int64  
 14  species_old    227433 non-null  object 
 15  flag_species   227433 non-null  int64  
dtypes: float64(4), int64(6), object(6)
memory usage: 27.8+ MB


In [5]:
str(JNorth1996_2020_df.columns.tolist())

"['id', 'sighting_date', 'species', 'number', 'latitude', 'longitude', 'flag_location', 'comments', 'school', 'grade', 'image_url', 'customerid', 'number_old', 'flag_number', 'species_old', 'flag_species']"

In [6]:
JNorth1996_2020_df.describe()

id        number       latitude      longitude  \
count  2.274330e+05  2.274330e+05  227433.000000  227433.000000   
mean   1.420650e+09  1.186529e+02      38.400690     -87.569965   
std    1.456887e+08  2.209080e+04       5.268651      10.880362   
min    8.508417e+08 -1.700000e+01     -12.100000    -165.107000   
25%    1.377751e+09  1.000000e+00      34.723000     -95.268000   
50%    1.477448e+09  1.000000e+00      39.503000     -86.787000   
75%    1.526383e+09  3.000000e+00      42.409000     -79.255000   
max    1.526525e+09  1.000000e+07      65.136000     -52.883000   

       flag_location          grade    customerid    number_old  \
count  227433.000000  227433.000000  2.274330e+05  2.274330e+05   
mean        0.598018 -907386.217597  9.451737e+06 -2.844270e+05   
std         0.490299  289894.342744  2.193961e+06  4.518136e+05   
min         0.000000 -999999.999000  0.000000e+00 -1.000000e+06   
25%         0.000000 -999999.999000  9.941050e+06 -1.000000e+06   
50%         1.000000 -999999.999000  9.977663e+06  1.000000e+00   
75%         1.000000 -999999.999000  1.000029e+07  3.000000e+00   
max         1.000000    1112.000000  1.004256e+07  1.000000e+07   

         flag_number   flag_species  
count  227433.000000  227433.000000  
mean        0.294816       0.130324  
std         0.467089       0.336660  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         1.000000       0.000000  
max         2.000000       1.000000

In [7]:
JNorth1996_2020_df.columns.tolist()

['id',
 'sighting_date',
 'species',
 'number',
 'latitude',
 'longitude',
 'flag_location',
 'comments',
 'school',
 'grade',
 'image_url',
 'customerid',
 'number_old',
 'flag_number',
 'species_old',
 'flag_species']

In [8]:
JNorth1996_2020_df.shape

(227433, 16)

In [9]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].unique())

[ 850841703  851824801  854409931 ... 1526524404 1526524405 1526525021]
['1996-12-16' '1996-12-06' '1996-09-21' ... '2020-12-29' '2020-12-30'
 '2020-12-31']
['Monarch Adult Sighted' 'Monarch Adult (FIRST sighted)'
 'Monarch Egg (FIRST sighted)' 'Monarch (OTHER Observations)'
 'Monarch Larva (FIRST sighted)' 'Monarch PEAK Migration'
 'Milkweed (FIRST sighted)' 'Monarch Fall Roost' 'Monarch Egg Sighted'
 'Monarch Larva Sighted' 'Monarch, Captive-Reared' 'Milkweed Sighted']
[       1      150       20       89       30     2000       50     7500
       23      200        5      300        4      100       71      120
      325       21       28     5000    20000       36       25        6
       54     1000      160       24       60       37        2     3000
  1000000       10  2000000       40       85       65       15        8
      500       14      112       75       80    15000       18    10000
      912   100000      400      800       12       35      600       16
    25000    

In [10]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].value_counts())

850841703     1
1526350827    1
1526350816    1
1526350817    1
1526350818    1
             ..
1432512422    1
1432519704    1
1432562236    1
1432562641    1
1526525021    1
Name: id, Length: 227433, dtype: int64
2018-08-22    392
2018-10-03    377
2018-09-06    353
2019-09-19    337
2018-09-12    335
             ... 
2015-12-31      1
2015-11-24      1
2015-11-22      1
2006-07-30      1
1996-12-16      1
Name: sighting_date, Length: 7491, dtype: int64
Monarch Adult Sighted            102306
Monarch Adult (FIRST sighted)     36409
Monarch Larva Sighted             17378
Monarch, Captive-Reared           14339
Monarch (OTHER Observations)      13581
Monarch Egg Sighted                8736
Milkweed (FIRST sighted)           8465
Monarch PEAK Migration             7689
Monarch Egg (FIRST sighted)        6876
Monarch Fall Roost                 5035
Monarch Larva (FIRST sighted)      4898
Milkweed Sighted                   1721
Name: species, dtype: int64
1       139319
2        24169
3

In [11]:
# Find null values
for column in JNorth1996_2020_df.columns:
   print(f"Column {column} has {JNorth1996_2020_df[column].isnull().sum()} null values")

Column id has 0 null values
Column sighting_date has 0 null values
Column species has 0 null values
Column number has 0 null values
Column latitude has 0 null values
Column longitude has 0 null values
Column flag_location has 0 null values
Column comments has 0 null values
Column school has 0 null values
Column grade has 0 null values
Column image_url has 0 null values
Column customerid has 0 null values
Column number_old has 0 null values
Column flag_number has 0 null values
Column species_old has 0 null values
Column flag_species has 0 null values


In [12]:
# Find duplicate entries
print(f"Duplicate entries: {JNorth1996_2020_df.duplicated().sum()}")

Duplicate entries: 0


In [13]:
# filtering by type of observation (Monarch Larva Sighted)
JNorth_Larva_Site_df = JNorth1996_2020_df[JNorth1996_2020_df['species']=='Monarch Larva Sighted']
JNorth_Larva_Site_df.head()

id sighting_date                species  number  latitude  \
48278  1344359409    2012-08-06  Monarch Larva Sighted       1    34.594   
48299  1344458076    2012-08-08  Monarch Larva Sighted       1    42.647   
48305  1344532312    2012-08-09  Monarch Larva Sighted      11    39.081   
48306  1344532445    2012-08-09  Monarch Larva Sighted       6    39.185   
48307  1344532566    2012-08-09  Monarch Larva Sighted       6    39.185   

       longitude  flag_location  \
48278   -112.700              0   
48299    -74.575              0   
48305    -76.558              0   
48306    -76.630              0   
48307    -76.630              0   

                                                comments  \
48278  Fiona Reid of Skull Valley reports her first m...   
48299           4th instar cat on milkweed ( incarnata).   
48305  Caterpillars spotted on common, poke, butterfl...   
48306  Caterpillars spotted on swamp and butterfly mi...   
48307  Caterpillars spotted on swamp and butterfly mi...   

                                        school       grade  \
48278                              -999999.999 -999999.999   
48299                              -999999.999 -999999.999   
48305  Arlington echo outdoor education center -999999.999   
48306  Arlington echo outdoor education center -999999.999   
48307  Arlington echo outdoor education center -999999.999   

                                               image_url  customerid  \
48278  https://journeynorth.org/jnorth/images/userfil...     9938243   
48299                                        -999999.999     9940596   
48305  https://journeynorth.org/jnorth/images/userfil...     9954266   
48306                                        -999999.999     9954266   
48307                                        -999999.999     9954266   

       number_old  flag_number            species_old  flag_species  
48278 -999999.999            1  Monarch Larva Sighted             0  
48299 -999999.999            1  Monarch Larva Sighted             0  
48305      11.000            0  Monarch Larva Sighted             0  
48306       6.000            0  Monarch Larva Sighted             0  
48307       6.000            0  Monarch Larva Sighted             0

In [14]:
JNorth_Larva_Site_df.to_csv('JNorth_Larva_Site.csv', header=True, mode='w' )

In [15]:
JNorth_Larva_Site_df.shape

(17378, 16)

# Format file for geoJSON 

In [16]:
JNorth_Larva_Site_df = pd.DataFrame(JNorth_Larva_Site_df)
JNorth_Larva_Site_df.drop(['id','flag_location', 'comments','school','grade','image_url','customerid','number_old','flag_number','species_old','flag_species'], axis=1, inplace=True)
JNorth_Larva_Site_df.head()

sighting_date                species  number  latitude  longitude
48278    2012-08-06  Monarch Larva Sighted       1    34.594   -112.700
48299    2012-08-08  Monarch Larva Sighted       1    42.647    -74.575
48305    2012-08-09  Monarch Larva Sighted      11    39.081    -76.558
48306    2012-08-09  Monarch Larva Sighted       6    39.185    -76.630
48307    2012-08-09  Monarch Larva Sighted       6    39.185    -76.630

In [17]:
cols = ['sighting_date', 'species', 'number', 'latitude', 'longitude']

In [18]:
JNorth_Larva_Site_df =JNorth_Larva_Site_df.groupby(['sighting_date', 'species', 'latitude', 'longitude']).sum().reset_index()
JNorth_Larva_Site_df.head()

sighting_date                species  latitude  longitude  number
0    2005-08-16  Monarch Larva Sighted    39.052    -77.125       5
1    2009-06-18  Monarch Larva Sighted    41.103    -95.958       2
2    2011-09-22  Monarch Larva Sighted    29.757    -95.365       1
3    2012-07-21  Monarch Larva Sighted    31.108    -83.422       4
4    2012-07-21  Monarch Larva Sighted    40.856    -74.229       9

In [19]:
JNorth_Larva_Site_df.columns.tolist()

['sighting_date', 'species', 'latitude', 'longitude', 'number']

In [20]:
JNorth_Larva_Site_df.head()

sighting_date                species  latitude  longitude  number
0    2005-08-16  Monarch Larva Sighted    39.052    -77.125       5
1    2009-06-18  Monarch Larva Sighted    41.103    -95.958       2
2    2011-09-22  Monarch Larva Sighted    29.757    -95.365       1
3    2012-07-21  Monarch Larva Sighted    31.108    -83.422       4
4    2012-07-21  Monarch Larva Sighted    40.856    -74.229       9

In [21]:
JNorth_Larva_Site_df.shape

(16798, 5)

In [22]:
JNorth_Larva_Site_year_df = JNorth_Larva_Site_df
JNorth_Larva_Site_year_df['year'] = pd.DatetimeIndex(JNorth_Larva_Site_df['sighting_date']).year

# 2001

In [23]:
JNorth_Larva_Site_2001_df = JNorth_Larva_Site_year_df[JNorth_Larva_Site_year_df['year']==2001]
JNorth_Larva_Site_2001_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, year]
Index: []

In [24]:
JNorth_Larva_Site_2001_df.shape

(0, 6)

In [25]:
JNorth_Larva_Site_2001_df = pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_df.drop(['year'], axis=1, inplace=True)

In [26]:
JNorth_Larva_Site_2001_df['month'] = pd.DatetimeIndex(JNorth_Larva_Site_2001_df['sighting_date']).month

## January

In [27]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_1_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==1]
JNorth_Larva_Site_2001_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [28]:
JNorth_Larva_Site_2001_1_df.shape

(0, 6)

In [29]:
JNorth_Larva_Site_2001_1_df = pd.DataFrame(JNorth_Larva_Site_2001_1_df)
JNorth_Larva_Site_2001_1_df.drop(['month'], axis=1, inplace=True)

In [30]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_1_df['latitude'] = JNorth_Larva_Site_2001_1_df['latitude'].astype(float)
JNorth_Larva_Site_2001_1_df['longitude'] = JNorth_Larva_Site_2001_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [31]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_1_df, cols)

In [32]:
file = open("JNorth_Larva_Site_2001_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [33]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_2_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==2]
JNorth_Larva_Site_2001_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [34]:
JNorth_Larva_Site_2001_2_df.shape

(0, 6)

In [35]:
JNorth_Larva_Site_2001_2_df= pd.DataFrame(JNorth_Larva_Site_2001_2_df)
JNorth_Larva_Site_2001_2_df.drop(['month'], axis=1, inplace=True)

In [36]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_2_df['latitude'] = JNorth_Larva_Site_2001_2_df['latitude'].astype(float)
JNorth_Larva_Site_2001_2_df['longitude'] = JNorth_Larva_Site_2001_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [37]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_2_df, cols)

In [38]:
file = open("JNorth_Larva_Site_2001_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [39]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_3_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==3]
JNorth_Larva_Site_2001_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [40]:
JNorth_Larva_Site_2001_3_df.shape

(0, 6)

In [41]:
JNorth_Larva_Site_2001_3_df =pd.DataFrame(JNorth_Larva_Site_2001_3_df)
JNorth_Larva_Site_2001_3_df.drop(['month'], axis=1, inplace=True)

In [42]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_3_df['latitude'] = JNorth_Larva_Site_2001_3_df['latitude'].astype(float)
JNorth_Larva_Site_2001_3_df['longitude'] = JNorth_Larva_Site_2001_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [43]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_3_df, cols)

In [44]:
file = open("JNorth_Larva_Site_2001_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [45]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_4_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==4]
JNorth_Larva_Site_2001_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [46]:
JNorth_Larva_Site_2001_4_df.shape

(0, 6)

In [47]:
JNorth_Larva_Site_2001_4_df = pd.DataFrame(JNorth_Larva_Site_2001_4_df)
JNorth_Larva_Site_2001_4_df.drop(['month'], axis=1, inplace=True)

In [48]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_4_df['latitude'] = JNorth_Larva_Site_2001_4_df['latitude'].astype(float)
JNorth_Larva_Site_2001_4_df['longitude'] = JNorth_Larva_Site_2001_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [49]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_4_df, cols)

In [50]:
file = open("JNorth_Larva_Site_2001_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [51]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_5_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==5]
JNorth_Larva_Site_2001_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [52]:
JNorth_Larva_Site_2001_5_df.shape

(0, 6)

In [53]:
JNorth_Larva_Site_2001_5_df =pd.DataFrame(JNorth_Larva_Site_2001_5_df)
JNorth_Larva_Site_2001_5_df.drop(['month'], axis=1, inplace=True)

In [54]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_5_df['latitude'] = JNorth_Larva_Site_2001_5_df['latitude'].astype(float)
JNorth_Larva_Site_2001_5_df['longitude'] = JNorth_Larva_Site_2001_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [55]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_5_df, cols)

In [56]:
file = open("JNorth_Larva_Site_2001_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [57]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_6_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==6]
JNorth_Larva_Site_2001_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [58]:
JNorth_Larva_Site_2001_6_df.shape

(0, 6)

In [59]:
JNorth_Larva_Site_2001_6_df=pd.DataFrame(JNorth_Larva_Site_2001_6_df)
JNorth_Larva_Site_2001_6_df.drop(['month'], axis=1, inplace=True)

In [60]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_6_df['latitude'] = JNorth_Larva_Site_2001_6_df['latitude'].astype(float)
JNorth_Larva_Site_2001_6_df['longitude'] = JNorth_Larva_Site_2001_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [61]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_6_df, cols)

In [62]:
file = open("JNorth_Larva_Site_2001_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [63]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_7_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==7]
JNorth_Larva_Site_2001_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [64]:
JNorth_Larva_Site_2001_7_df.shape

(0, 6)

In [65]:
JNorth_Larva_Site_2001_7_df = pd.DataFrame(JNorth_Larva_Site_2001_7_df)
JNorth_Larva_Site_2001_7_df.drop(['month'], axis=1, inplace=True)

In [66]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_7_df['latitude'] = JNorth_Larva_Site_2001_7_df['latitude'].astype(float)
JNorth_Larva_Site_2001_7_df['longitude'] = JNorth_Larva_Site_2001_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [67]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_7_df, cols)

In [68]:
file = open("JNorth_Larva_Site_2001_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [69]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_8_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==8]
JNorth_Larva_Site_2001_8_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [70]:
JNorth_Larva_Site_2001_8_df.shape

(0, 6)

In [71]:
JNorth_Larva_Site_2001_8_df = pd.DataFrame(JNorth_Larva_Site_2001_8_df)
JNorth_Larva_Site_2001_8_df.drop(['month'], axis=1, inplace=True)

In [72]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_8_df['latitude'] = JNorth_Larva_Site_2001_8_df['latitude'].astype(float)
JNorth_Larva_Site_2001_8_df['longitude'] = JNorth_Larva_Site_2001_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [73]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_8_df, cols)

In [74]:
file = open("JNorth_Larva_Site_2001_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [75]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_9_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==9]
JNorth_Larva_Site_2001_9_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [76]:
JNorth_Larva_Site_2001_9_df.shape

(0, 6)

In [77]:
JNorth_Larva_Site_2001_9_df = pd.DataFrame(JNorth_Larva_Site_2001_9_df)
JNorth_Larva_Site_2001_9_df.drop(['month'], axis=1, inplace=True)

In [78]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_9_df['latitude'] = JNorth_Larva_Site_2001_9_df['latitude'].astype(float)
JNorth_Larva_Site_2001_9_df['longitude'] = JNorth_Larva_Site_2001_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [79]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_9_df, cols)

In [80]:
file = open("JNorth_Larva_Site_2001_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [81]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_10_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==10]
JNorth_Larva_Site_2001_10_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [82]:
JNorth_Larva_Site_2001_10_df.shape

(0, 6)

In [83]:
JNorth_Larva_Site_2001_10_df = pd.DataFrame(JNorth_Larva_Site_2001_10_df)
JNorth_Larva_Site_2001_10_df.drop(['month'], axis=1, inplace=True)

In [84]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_10_df['latitude'] = JNorth_Larva_Site_2001_10_df['latitude'].astype(float)
JNorth_Larva_Site_2001_10_df['longitude'] = JNorth_Larva_Site_2001_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [85]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_10_df, cols)

In [86]:
file = open("JNorth_Larva_Site_2001_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [87]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_11_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==11]
JNorth_Larva_Site_2001_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [88]:
JNorth_Larva_Site_2001_11_df.shape

(0, 6)

In [89]:
JNorth_Larva_Site_2001_11_df = pd.DataFrame(JNorth_Larva_Site_2001_11_df)
JNorth_Larva_Site_2001_11_df.drop(['month'], axis=1, inplace=True)

In [90]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_11_df['latitude'] = JNorth_Larva_Site_2001_11_df['latitude'].astype(float)
JNorth_Larva_Site_2001_11_df['longitude'] = JNorth_Larva_Site_2001_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [91]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_11_df, cols)

In [92]:
file = open("JNorth_Larva_Site_2001_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [93]:
JNorth_Larva_Site_2001_month_df =pd.DataFrame(JNorth_Larva_Site_2001_df)
JNorth_Larva_Site_2001_12_df = JNorth_Larva_Site_2001_month_df[JNorth_Larva_Site_2001_month_df['month']==12]
JNorth_Larva_Site_2001_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [94]:
JNorth_Larva_Site_2001_12_df.shape

(0, 6)

In [95]:
JNorth_Larva_Site_2001_12_df = pd.DataFrame(JNorth_Larva_Site_2001_12_df)
JNorth_Larva_Site_2001_12_df.drop(['month'], axis=1, inplace=True)

In [96]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2001_12_df['latitude'] = JNorth_Larva_Site_2001_12_df['latitude'].astype(float)
JNorth_Larva_Site_2001_12_df['longitude'] = JNorth_Larva_Site_2001_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [97]:
geojson = df_to_geojson(JNorth_Larva_Site_2001_12_df, cols)

In [98]:
file = open("JNorth_Larva_Site_2001_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2002

In [99]:
JNorth_Larva_Site_2002_df = JNorth_Larva_Site_year_df[JNorth_Larva_Site_year_df['year']==2002]
JNorth_Larva_Site_2002_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, year]
Index: []

In [100]:
JNorth_Larva_Site_2002_df.shape

(0, 6)

In [101]:
JNorth_Larva_Site_2002_df = pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_df.drop(['year'], axis=1, inplace=True)

In [102]:
JNorth_Larva_Site_2002_df.tail

<bound method NDFrame.tail of Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number]
Index: []>

In [103]:
JNorth_Larva_Site_2002_df['month'] = pd.DatetimeIndex(JNorth_Larva_Site_2002_df['sighting_date']).month

## January

In [104]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_1_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==1]
JNorth_Larva_Site_2002_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [105]:
JNorth_Larva_Site_2002_1_df.shape

(0, 6)

In [106]:
JNorth_Larva_Site_2002_1_df = pd.DataFrame(JNorth_Larva_Site_2002_1_df)
JNorth_Larva_Site_2002_1_df.drop(['month'], axis=1, inplace=True)

In [107]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_1_df['latitude'] = JNorth_Larva_Site_2002_1_df['latitude'].astype(float)
JNorth_Larva_Site_2002_1_df['longitude'] = JNorth_Larva_Site_2002_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [108]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_1_df, cols)

In [109]:
file = open("JNorth_Larva_Site_2002_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [110]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_2_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==2]
JNorth_Larva_Site_2002_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [111]:
JNorth_Larva_Site_2002_2_df.shape

(0, 6)

In [112]:
JNorth_Larva_Site_2002_2_df = pd.DataFrame(JNorth_Larva_Site_2002_2_df)
JNorth_Larva_Site_2002_2_df.drop(['month'], axis=1, inplace=True)

In [113]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_2_df['latitude'] = JNorth_Larva_Site_2002_2_df['latitude'].astype(float)
JNorth_Larva_Site_2002_2_df['longitude'] = JNorth_Larva_Site_2002_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [114]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_2_df, cols)

In [115]:
file = open("JNorth_Larva_Site_2002_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [116]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_3_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==3]
JNorth_Larva_Site_2002_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [117]:
JNorth_Larva_Site_2002_3_df.shape

(0, 6)

In [118]:
JNorth_Larva_Site_2002_3_df = pd.DataFrame(JNorth_Larva_Site_2002_3_df)
JNorth_Larva_Site_2002_3_df.drop(['month'], axis=1, inplace=True)

In [119]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_3_df['latitude'] = JNorth_Larva_Site_2002_3_df['latitude'].astype(float)
JNorth_Larva_Site_2002_3_df['longitude'] = JNorth_Larva_Site_2002_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [120]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_3_df, cols)

In [121]:
file = open("JNorth_Larva_Site_2002_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [122]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_4_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==4]
JNorth_Larva_Site_2002_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [123]:
JNorth_Larva_Site_2002_4_df.shape

(0, 6)

In [124]:
JNorth_Larva_Site_2002_4_df = pd.DataFrame(JNorth_Larva_Site_2002_4_df)
JNorth_Larva_Site_2002_4_df.drop(['month'], axis=1, inplace=True)

In [125]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_4_df['latitude'] = JNorth_Larva_Site_2002_4_df['latitude'].astype(float)
JNorth_Larva_Site_2002_4_df['longitude'] = JNorth_Larva_Site_2002_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [126]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_4_df, cols)

In [127]:
file = open("JNorth_Larva_Site_2002_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [128]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_5_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==5]
JNorth_Larva_Site_2002_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [129]:
JNorth_Larva_Site_2002_5_df.shape

(0, 6)

In [130]:
JNorth_Larva_Site_2002_5_df = pd.DataFrame(JNorth_Larva_Site_2002_5_df)
JNorth_Larva_Site_2002_5_df.drop(['month'], axis=1, inplace=True)

In [131]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_5_df['latitude'] = JNorth_Larva_Site_2002_5_df['latitude'].astype(float)
JNorth_Larva_Site_2002_5_df['longitude'] = JNorth_Larva_Site_2002_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [132]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_5_df, cols)

In [133]:
file = open("JNorth_Larva_Site_2002_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [134]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_6_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==6]
JNorth_Larva_Site_2002_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [135]:
JNorth_Larva_Site_2002_6_df.shape

(0, 6)

In [136]:
JNorth_Larva_Site_2002_6_df = pd.DataFrame(JNorth_Larva_Site_2002_6_df)
JNorth_Larva_Site_2002_6_df.drop(['month'], axis=1, inplace=True)

In [137]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_6_df['latitude'] = JNorth_Larva_Site_2002_6_df['latitude'].astype(float)
JNorth_Larva_Site_2002_6_df['longitude'] = JNorth_Larva_Site_2002_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [138]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_6_df, cols)

In [139]:
file = open("JNorth_Larva_Site_2002_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [140]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_7_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==7]
JNorth_Larva_Site_2002_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [141]:
JNorth_Larva_Site_2002_7_df.shape

(0, 6)

In [142]:
JNorth_Larva_Site_2002_7_df = pd.DataFrame(JNorth_Larva_Site_2002_7_df)
JNorth_Larva_Site_2002_7_df.drop(['month'], axis=1, inplace=True)

In [143]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_7_df['latitude'] = JNorth_Larva_Site_2002_7_df['latitude'].astype(float)
JNorth_Larva_Site_2002_7_df['longitude'] = JNorth_Larva_Site_2002_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [144]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_7_df, cols)

In [145]:
file = open("JNorth_Larva_Site_2002_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [146]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_8_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==8]
JNorth_Larva_Site_2002_8_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [147]:
JNorth_Larva_Site_2002_8_df.shape

(0, 6)

In [148]:
JNorth_Larva_Site_2002_8_df = pd.DataFrame(JNorth_Larva_Site_2002_8_df)
JNorth_Larva_Site_2002_8_df.drop(['month'], axis=1, inplace=True)

In [149]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_8_df['latitude'] = JNorth_Larva_Site_2002_8_df['latitude'].astype(float)
JNorth_Larva_Site_2002_8_df['longitude'] = JNorth_Larva_Site_2002_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [150]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_8_df, cols)

In [151]:
file = open("JNorth_Larva_Site_2002_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [152]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_9_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==9]
JNorth_Larva_Site_2002_9_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [153]:
JNorth_Larva_Site_2002_9_df.shape

(0, 6)

In [154]:
JNorth_Larva_Site_2002_9_df = pd.DataFrame(JNorth_Larva_Site_2002_9_df)
JNorth_Larva_Site_2002_9_df.drop(['month'], axis=1, inplace=True)

In [155]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_9_df['latitude'] = JNorth_Larva_Site_2002_9_df['latitude'].astype(float)
JNorth_Larva_Site_2002_9_df['longitude'] = JNorth_Larva_Site_2002_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [156]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_9_df, cols)

In [157]:
file = open("JNorth_Larva_Site_2002_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [158]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_10_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==10]
JNorth_Larva_Site_2002_10_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [159]:
JNorth_Larva_Site_2002_10_df.shape

(0, 6)

In [160]:
JNorth_Larva_Site_2002_10_df = pd.DataFrame(JNorth_Larva_Site_2002_10_df)
JNorth_Larva_Site_2002_10_df.drop(['month'], axis=1, inplace=True)

In [161]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_10_df['latitude'] = JNorth_Larva_Site_2002_10_df['latitude'].astype(float)
JNorth_Larva_Site_2002_10_df['longitude'] = JNorth_Larva_Site_2002_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [162]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_10_df, cols)

In [163]:
file = open("JNorth_Larva_Site_2002_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [164]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_11_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==11]
JNorth_Larva_Site_2002_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [165]:
JNorth_Larva_Site_2002_11_df.shape

(0, 6)

In [166]:
JNorth_Larva_Site_2002_11_df = pd.DataFrame(JNorth_Larva_Site_2002_11_df)
JNorth_Larva_Site_2002_11_df.drop(['month'], axis=1, inplace=True)

In [167]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_11_df['latitude'] = JNorth_Larva_Site_2002_11_df['latitude'].astype(float)
JNorth_Larva_Site_2002_11_df['longitude'] = JNorth_Larva_Site_2002_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [168]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_11_df, cols)

In [169]:
file = open("JNorth_Larva_Site_2002_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [170]:
JNorth_Larva_Site_2002_month_df =pd.DataFrame(JNorth_Larva_Site_2002_df)
JNorth_Larva_Site_2002_12_df = JNorth_Larva_Site_2002_month_df[JNorth_Larva_Site_2002_month_df['month']==12]
JNorth_Larva_Site_2002_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [171]:
JNorth_Larva_Site_2002_12_df.shape

(0, 6)

In [172]:
JNorth_Larva_Site_2002_12_df = pd.DataFrame(JNorth_Larva_Site_2002_12_df)
JNorth_Larva_Site_2002_12_df.drop(['month'], axis=1, inplace=True)

In [173]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2002_12_df['latitude'] = JNorth_Larva_Site_2002_12_df['latitude'].astype(float)
JNorth_Larva_Site_2002_12_df['longitude'] = JNorth_Larva_Site_2002_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [174]:
geojson = df_to_geojson(JNorth_Larva_Site_2002_12_df, cols)

In [175]:
file = open("JNorth_Larva_Site_2002_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2003

In [176]:
JNorth_Larva_Site_2003_df = JNorth_Larva_Site_year_df[JNorth_Larva_Site_year_df['year']==2003]
JNorth_Larva_Site_2003_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, year]
Index: []

In [177]:
JNorth_Larva_Site_2003_df.shape

(0, 6)

In [178]:
JNorth_Larva_Site_2003_df = pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_df.drop(['year'], axis=1, inplace=True)

In [179]:
JNorth_Larva_Site_2003_df.tail

<bound method NDFrame.tail of Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number]
Index: []>

In [180]:
JNorth_Larva_Site_2003_df['month'] = pd.DatetimeIndex(JNorth_Larva_Site_2003_df['sighting_date']).month

## January

In [181]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_1_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==1]
JNorth_Larva_Site_2003_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [182]:
JNorth_Larva_Site_2003_1_df.shape

(0, 6)

In [183]:
JNorth_Larva_Site_2003_1_df = pd.DataFrame(JNorth_Larva_Site_2003_1_df)
JNorth_Larva_Site_2003_1_df.drop(['month'], axis=1, inplace=True)

In [184]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_1_df['latitude'] = JNorth_Larva_Site_2003_1_df['latitude'].astype(float)
JNorth_Larva_Site_2003_1_df['longitude'] = JNorth_Larva_Site_2003_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [185]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_1_df, cols)

In [186]:
file = open("JNorth_Larva_Site_2003_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [187]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_2_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==2]
JNorth_Larva_Site_2003_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [188]:
JNorth_Larva_Site_2003_2_df.shape

(0, 6)

In [189]:
JNorth_Larva_Site_2003_2_df = pd.DataFrame(JNorth_Larva_Site_2003_2_df)
JNorth_Larva_Site_2003_2_df.drop(['month'], axis=1, inplace=True)

In [190]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_2_df['latitude'] = JNorth_Larva_Site_2003_2_df['latitude'].astype(float)
JNorth_Larva_Site_2003_2_df['longitude'] = JNorth_Larva_Site_2003_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [191]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_2_df, cols)

In [192]:
file = open("JNorth_Larva_Site_2003_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [193]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_3_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==3]
JNorth_Larva_Site_2003_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [194]:
JNorth_Larva_Site_2003_3_df.shape

(0, 6)

In [195]:
JNorth_Larva_Site_2003_3_df = pd.DataFrame(JNorth_Larva_Site_2003_3_df)
JNorth_Larva_Site_2003_3_df.drop(['month'], axis=1, inplace=True)

In [196]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_3_df['latitude'] = JNorth_Larva_Site_2003_3_df['latitude'].astype(float)
JNorth_Larva_Site_2003_3_df['longitude'] = JNorth_Larva_Site_2003_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [197]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_3_df, cols)

In [198]:
file = open("JNorth_Larva_Site_2003_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [199]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_4_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==4]
JNorth_Larva_Site_2003_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [200]:
JNorth_Larva_Site_2003_4_df.shape

(0, 6)

In [201]:
JNorth_Larva_Site_2003_4_df =pd.DataFrame(JNorth_Larva_Site_2003_4_df)
JNorth_Larva_Site_2003_4_df.drop(['month'], axis=1, inplace=True)

In [202]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_4_df['latitude'] = JNorth_Larva_Site_2003_4_df['latitude'].astype(float)
JNorth_Larva_Site_2003_4_df['longitude'] = JNorth_Larva_Site_2003_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [203]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_4_df, cols)

In [204]:
file = open("JNorth_Larva_Site_2003_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [205]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_5_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==5]
JNorth_Larva_Site_2003_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [206]:
JNorth_Larva_Site_2003_5_df.shape

(0, 6)

In [207]:
JNorth_Larva_Site_2003_5_df =pd.DataFrame(JNorth_Larva_Site_2003_5_df)
JNorth_Larva_Site_2003_5_df.drop(['month'], axis=1, inplace=True)

In [208]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_5_df['latitude'] = JNorth_Larva_Site_2003_5_df['latitude'].astype(float)
JNorth_Larva_Site_2003_5_df['longitude'] = JNorth_Larva_Site_2003_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [209]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_5_df, cols)

In [210]:
file = open("JNorth_Larva_Site_2003_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [211]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_6_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==6]
JNorth_Larva_Site_2003_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [212]:
JNorth_Larva_Site_2003_6_df.shape

(0, 6)

In [213]:
JNorth_Larva_Site_2003_6_df = pd.DataFrame(JNorth_Larva_Site_2003_6_df)
JNorth_Larva_Site_2003_6_df.drop(['month'], axis=1, inplace=True)

In [214]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_6_df['latitude'] = JNorth_Larva_Site_2003_6_df['latitude'].astype(float)
JNorth_Larva_Site_2003_6_df['longitude'] = JNorth_Larva_Site_2003_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [215]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_6_df, cols)

In [216]:
file = open("JNorth_Larva_Site_2003_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [217]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_7_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==7]
JNorth_Larva_Site_2003_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [218]:
JNorth_Larva_Site_2003_7_df.shape

(0, 6)

In [219]:
JNorth_Larva_Site_2003_7_df = pd.DataFrame(JNorth_Larva_Site_2003_7_df)
JNorth_Larva_Site_2003_7_df.drop(['month'], axis=1, inplace=True)

In [220]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_7_df['latitude'] = JNorth_Larva_Site_2003_7_df['latitude'].astype(float)
JNorth_Larva_Site_2003_7_df['longitude'] = JNorth_Larva_Site_2003_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [221]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_7_df, cols)

In [222]:
file = open("JNorth_Larva_Site_2003_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [223]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_8_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==8]
JNorth_Larva_Site_2003_8_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [224]:
JNorth_Larva_Site_2003_8_df.shape

(0, 6)

In [225]:
JNorth_Larva_Site_2003_8_df = pd.DataFrame(JNorth_Larva_Site_2003_8_df)
JNorth_Larva_Site_2003_8_df.drop(['month'], axis=1, inplace=True)

In [226]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_8_df['latitude'] = JNorth_Larva_Site_2003_8_df['latitude'].astype(float)
JNorth_Larva_Site_2003_8_df['longitude'] = JNorth_Larva_Site_2003_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [227]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_8_df, cols)

In [228]:
file = open("JNorth_Larva_Site_2003_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [229]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_9_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==9]
JNorth_Larva_Site_2003_9_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [230]:
JNorth_Larva_Site_2003_9_df.shape

(0, 6)

In [231]:
JNorth_Larva_Site_2003_9_df = pd.DataFrame(JNorth_Larva_Site_2003_9_df)
JNorth_Larva_Site_2003_9_df.drop(['month'], axis=1, inplace=True)

In [232]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_9_df['latitude'] = JNorth_Larva_Site_2003_9_df['latitude'].astype(float)
JNorth_Larva_Site_2003_9_df['longitude'] = JNorth_Larva_Site_2003_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [233]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_9_df, cols)

In [234]:
file = open("JNorth_Larva_Site_2003_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [235]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_10_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==10]
JNorth_Larva_Site_2003_10_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [236]:
JNorth_Larva_Site_2003_10_df.shape

(0, 6)

In [237]:
JNorth_Larva_Site_2003_10_df = pd.DataFrame(JNorth_Larva_Site_2003_10_df)
JNorth_Larva_Site_2003_10_df.drop(['month'], axis=1, inplace=True)

In [238]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_10_df['latitude'] = JNorth_Larva_Site_2003_10_df['latitude'].astype(float)
JNorth_Larva_Site_2003_10_df['longitude'] = JNorth_Larva_Site_2003_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [239]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_10_df, cols)

In [240]:
file = open("JNorth_Larva_Site_2003_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [241]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_11_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==11]
JNorth_Larva_Site_2003_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [242]:
JNorth_Larva_Site_2003_11_df.shape

(0, 6)

In [243]:
JNorth_Larva_Site_2003_11_df = pd.DataFrame(JNorth_Larva_Site_2003_11_df)
JNorth_Larva_Site_2003_11_df.drop(['month'], axis=1, inplace=True)

In [244]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_11_df['latitude'] = JNorth_Larva_Site_2003_11_df['latitude'].astype(float)
JNorth_Larva_Site_2003_11_df['longitude'] = JNorth_Larva_Site_2003_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [245]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_11_df, cols)

In [246]:
file = open("JNorth_Larva_Site_2003_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [247]:
JNorth_Larva_Site_2003_month_df =pd.DataFrame(JNorth_Larva_Site_2003_df)
JNorth_Larva_Site_2003_12_df = JNorth_Larva_Site_2003_month_df[JNorth_Larva_Site_2003_month_df['month']==12]
JNorth_Larva_Site_2003_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [248]:
JNorth_Larva_Site_2003_12_df.shape

(0, 6)

In [249]:
JNorth_Larva_Site_2003_12_df = pd.DataFrame(JNorth_Larva_Site_2003_12_df)
JNorth_Larva_Site_2003_12_df.drop(['month'], axis=1, inplace=True)

In [250]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2003_12_df['latitude'] = JNorth_Larva_Site_2003_12_df['latitude'].astype(float)
JNorth_Larva_Site_2003_12_df['longitude'] = JNorth_Larva_Site_2003_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [251]:
geojson = df_to_geojson(JNorth_Larva_Site_2003_12_df, cols)

In [252]:
file = open("JNorth_Larva_Site_2003_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2004

In [253]:
JNorth_Larva_Site_2004_df = JNorth_Larva_Site_year_df[JNorth_Larva_Site_year_df['year']==2004]
JNorth_Larva_Site_2004_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, year]
Index: []

In [254]:
JNorth_Larva_Site_2004_df.shape

(0, 6)

In [255]:
JNorth_Larva_Site_2004_df = pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_df.drop(['year'], axis=1, inplace=True)

In [256]:
JNorth_Larva_Site_2004_df.tail

<bound method NDFrame.tail of Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number]
Index: []>

In [257]:
JNorth_Larva_Site_2004_df['month'] = pd.DatetimeIndex(JNorth_Larva_Site_2004_df['sighting_date']).month

## January

In [258]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_1_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==1]
JNorth_Larva_Site_2004_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [259]:
JNorth_Larva_Site_2004_1_df.shape

(0, 6)

In [260]:
JNorth_Larva_Site_2004_1_df = pd.DataFrame(JNorth_Larva_Site_2004_1_df)
JNorth_Larva_Site_2004_1_df.drop(['month'], axis=1, inplace=True)

In [261]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_1_df['latitude'] = JNorth_Larva_Site_2004_1_df['latitude'].astype(float)
JNorth_Larva_Site_2004_1_df['longitude'] = JNorth_Larva_Site_2004_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [262]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_1_df, cols)

In [263]:
file = open("JNorth_Larva_Site_2004_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [264]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_2_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==2]
JNorth_Larva_Site_2004_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [265]:
JNorth_Larva_Site_2004_2_df.shape

(0, 6)

In [266]:
JNorth_Larva_Site_2004_2_df = pd.DataFrame(JNorth_Larva_Site_2004_2_df)
JNorth_Larva_Site_2004_2_df.drop(['month'], axis=1, inplace=True)

In [267]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_2_df['latitude'] = JNorth_Larva_Site_2004_2_df['latitude'].astype(float)
JNorth_Larva_Site_2004_2_df['longitude'] = JNorth_Larva_Site_2004_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [268]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_2_df, cols)

In [269]:
file = open("JNorth_Larva_Site_2004_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [270]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_3_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==3]
JNorth_Larva_Site_2004_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [271]:
JNorth_Larva_Site_2004_3_df.shape

(0, 6)

In [272]:
JNorth_Larva_Site_2004_3_df = pd.DataFrame(JNorth_Larva_Site_2004_3_df)
JNorth_Larva_Site_2004_3_df.drop(['month'], axis=1, inplace=True)

In [273]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_3_df['latitude'] = JNorth_Larva_Site_2004_3_df['latitude'].astype(float)
JNorth_Larva_Site_2004_3_df['longitude'] = JNorth_Larva_Site_2004_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [274]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_3_df, cols)

In [275]:
file = open("JNorth_Larva_Site_2004_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [276]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_4_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==4]
JNorth_Larva_Site_2004_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [277]:
JNorth_Larva_Site_2004_4_df.shape

(0, 6)

In [278]:
JNorth_Larva_Site_2004_4_df = pd.DataFrame(JNorth_Larva_Site_2004_4_df)
JNorth_Larva_Site_2004_4_df.drop(['month'], axis=1, inplace=True)

In [279]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_4_df['latitude'] = JNorth_Larva_Site_2004_4_df['latitude'].astype(float)
JNorth_Larva_Site_2004_4_df['longitude'] = JNorth_Larva_Site_2004_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [280]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_4_df, cols)

In [281]:
file = open("JNorth_Larva_Site_2004_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [282]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_5_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==5]
JNorth_Larva_Site_2004_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [283]:
JNorth_Larva_Site_2004_5_df.shape

(0, 6)

In [284]:
JNorth_Larva_Site_2004_5_df = pd.DataFrame(JNorth_Larva_Site_2004_5_df)
JNorth_Larva_Site_2004_5_df.drop(['month'], axis=1, inplace=True)

In [285]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_5_df['latitude'] = JNorth_Larva_Site_2004_5_df['latitude'].astype(float)
JNorth_Larva_Site_2004_5_df['longitude'] = JNorth_Larva_Site_2004_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [286]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_5_df, cols)

In [287]:
file = open("JNorth_Larva_Site_2004_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [288]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_6_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==6]
JNorth_Larva_Site_2004_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [289]:
JNorth_Larva_Site_2004_6_df.shape

(0, 6)

In [290]:
JNorth_Larva_Site_2004_6_df = pd.DataFrame(JNorth_Larva_Site_2004_6_df)
JNorth_Larva_Site_2004_6_df.drop(['month'], axis=1, inplace=True)

In [291]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_6_df['latitude'] = JNorth_Larva_Site_2004_6_df['latitude'].astype(float)
JNorth_Larva_Site_2004_6_df['longitude'] = JNorth_Larva_Site_2004_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [292]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_6_df, cols)

In [293]:
file = open("JNorth_Larva_Site_2004_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [294]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_7_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==7]
JNorth_Larva_Site_2004_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [295]:
JNorth_Larva_Site_2004_7_df.shape

(0, 6)

In [296]:
JNorth_Larva_Site_2004_7_df = pd.DataFrame(JNorth_Larva_Site_2004_7_df)
JNorth_Larva_Site_2004_7_df.drop(['month'], axis=1, inplace=True)

In [297]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_7_df['latitude'] = JNorth_Larva_Site_2004_7_df['latitude'].astype(float)
JNorth_Larva_Site_2004_7_df['longitude'] = JNorth_Larva_Site_2004_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [298]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_7_df, cols)

In [299]:
file = open("JNorth_Larva_Site_2004_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [300]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_8_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==8]
JNorth_Larva_Site_2004_8_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [301]:
JNorth_Larva_Site_2004_8_df.shape

(0, 6)

In [302]:
JNorth_Larva_Site_2004_8_df= pd.DataFrame(JNorth_Larva_Site_2004_8_df)
JNorth_Larva_Site_2004_8_df.drop(['month'], axis=1, inplace=True)

In [303]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_8_df['latitude'] = JNorth_Larva_Site_2004_8_df['latitude'].astype(float)
JNorth_Larva_Site_2004_8_df['longitude'] = JNorth_Larva_Site_2004_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [304]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_8_df, cols)

In [305]:
file = open("JNorth_Larva_Site_2004_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [306]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_9_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==9]
JNorth_Larva_Site_2004_9_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [307]:
JNorth_Larva_Site_2004_9_df.shape

(0, 6)

In [308]:
JNorth_Larva_Site_2004_9_df = pd.DataFrame(JNorth_Larva_Site_2004_9_df)
JNorth_Larva_Site_2004_9_df.drop(['month'], axis=1, inplace=True)

In [309]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_9_df['latitude'] = JNorth_Larva_Site_2004_9_df['latitude'].astype(float)
JNorth_Larva_Site_2004_9_df['longitude'] = JNorth_Larva_Site_2004_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [310]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_9_df, cols)

In [311]:
file = open("JNorth_Larva_Site_2004_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [312]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_10_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==10]
JNorth_Larva_Site_2004_10_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [313]:
JNorth_Larva_Site_2004_10_df.shape

(0, 6)

In [314]:
JNorth_Larva_Site_2004_10_df = pd.DataFrame(JNorth_Larva_Site_2004_10_df)
JNorth_Larva_Site_2004_10_df.drop(['month'], axis=1, inplace=True)

In [315]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_10_df['latitude'] = JNorth_Larva_Site_2004_10_df['latitude'].astype(float)
JNorth_Larva_Site_2004_10_df['longitude'] = JNorth_Larva_Site_2004_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [316]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_10_df, cols)

In [317]:
file = open("JNorth_Larva_Site_2004_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [318]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_11_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==11]
JNorth_Larva_Site_2004_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [319]:
JNorth_Larva_Site_2004_11_df.shape

(0, 6)

In [320]:
JNorth_Larva_Site_2004_11_df = pd.DataFrame(JNorth_Larva_Site_2004_11_df)
JNorth_Larva_Site_2004_11_df.drop(['month'], axis=1, inplace=True)

In [321]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_11_df['latitude'] = JNorth_Larva_Site_2004_11_df['latitude'].astype(float)
JNorth_Larva_Site_2004_11_df['longitude'] = JNorth_Larva_Site_2004_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [322]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_11_df, cols)

In [323]:
file = open("JNorth_Larva_Site_2004_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [324]:
JNorth_Larva_Site_2004_month_df =pd.DataFrame(JNorth_Larva_Site_2004_df)
JNorth_Larva_Site_2004_12_df = JNorth_Larva_Site_2004_month_df[JNorth_Larva_Site_2004_month_df['month']==12]
JNorth_Larva_Site_2004_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [325]:
JNorth_Larva_Site_2004_12_df.shape

(0, 6)

In [326]:
JNorth_Larva_Site_2004_12_df = pd.DataFrame(JNorth_Larva_Site_2004_12_df)
JNorth_Larva_Site_2004_12_df.drop(['month'], axis=1, inplace=True)

In [327]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2004_12_df['latitude'] = JNorth_Larva_Site_2004_12_df['latitude'].astype(float)
JNorth_Larva_Site_2004_12_df['longitude'] = JNorth_Larva_Site_2004_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [328]:
geojson = df_to_geojson(JNorth_Larva_Site_2004_12_df, cols)

In [329]:
file = open("JNorth_Larva_Site_2004_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2005

In [330]:
JNorth_Larva_Site_2005_df = JNorth_Larva_Site_df[JNorth_Larva_Site_df['year']==2005]
JNorth_Larva_Site_2005_df.tail()

sighting_date                species  latitude  longitude  number  year
0    2005-08-16  Monarch Larva Sighted    39.052    -77.125       5  2005

In [331]:
JNorth_Larva_Site_2005_df.shape

(1, 6)

In [332]:
JNorth_Larva_Site_2005_df = pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_df.drop(['year'], axis=1, inplace=True)

In [333]:
JNorth_Larva_Site_2005_df.tail

<bound method NDFrame.tail of   sighting_date                species  latitude  longitude  number
0    2005-08-16  Monarch Larva Sighted    39.052    -77.125       5>

In [334]:
JNorth_Larva_Site_2005_df['month'] = pd.DatetimeIndex(JNorth_Larva_Site_2005_df['sighting_date']).month

## January

In [335]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_1_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==1]
JNorth_Larva_Site_2005_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [336]:
JNorth_Larva_Site_2005_1_df.shape

(0, 6)

In [337]:
JNorth_Larva_Site_2005_1_df = pd.DataFrame(JNorth_Larva_Site_2005_1_df)
JNorth_Larva_Site_2005_1_df.drop(['month'], axis=1, inplace=True)

In [338]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_1_df['latitude'] = JNorth_Larva_Site_2005_1_df['latitude'].astype(float)
JNorth_Larva_Site_2005_1_df['longitude'] = JNorth_Larva_Site_2005_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [339]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_1_df, cols)

In [340]:
file = open("JNorth_Larva_Site_2005_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# February

In [341]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_2_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==2]
JNorth_Larva_Site_2005_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [342]:
JNorth_Larva_Site_2005_2_df.shape

(0, 6)

In [343]:
JNorth_Larva_Site_2005_2_df = pd.DataFrame(JNorth_Larva_Site_2005_2_df)
JNorth_Larva_Site_2005_2_df.drop(['month'], axis=1, inplace=True)

In [344]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_2_df['latitude'] = JNorth_Larva_Site_2005_2_df['latitude'].astype(float)
JNorth_Larva_Site_2005_2_df['longitude'] = JNorth_Larva_Site_2005_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [345]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_2_df, cols)

In [346]:
file = open("JNorth_Larva_Site_2005_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# March

In [347]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_3_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==3]
JNorth_Larva_Site_2005_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [348]:
JNorth_Larva_Site_2005_3_df.shape

(0, 6)

In [349]:
JNorth_Larva_Site_2005_3_df = pd.DataFrame(JNorth_Larva_Site_2005_3_df)
JNorth_Larva_Site_2005_3_df.drop(['month'], axis=1, inplace=True)

In [350]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_3_df['latitude'] = JNorth_Larva_Site_2005_3_df['latitude'].astype(float)
JNorth_Larva_Site_2005_3_df['longitude'] = JNorth_Larva_Site_2005_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [351]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_3_df, cols)

In [352]:
file = open("JNorth_Larva_Site_2005_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# April

In [353]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_4_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==4]
JNorth_Larva_Site_2005_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [354]:
JNorth_Larva_Site_2005_4_df.shape

(0, 6)

In [355]:
JNorth_Larva_Site_2005_4_df= pd.DataFrame(JNorth_Larva_Site_2005_4_df)
JNorth_Larva_Site_2005_4_df.drop(['month'], axis=1, inplace=True)

In [356]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_4_df['latitude'] = JNorth_Larva_Site_2005_4_df['latitude'].astype(float)
JNorth_Larva_Site_2005_4_df['longitude'] = JNorth_Larva_Site_2005_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [357]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_4_df, cols)

In [358]:
file = open("JNorth_Larva_Site_2005_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# May

In [359]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_5_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==5]
JNorth_Larva_Site_2005_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [360]:
JNorth_Larva_Site_2005_5_df.shape

(0, 6)

In [361]:
JNorth_Larva_Site_2005_5_df = pd.DataFrame(JNorth_Larva_Site_2005_5_df)
JNorth_Larva_Site_2005_5_df.drop(['month'], axis=1, inplace=True)

In [362]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_5_df['latitude'] = JNorth_Larva_Site_2005_5_df['latitude'].astype(float)
JNorth_Larva_Site_2005_5_df['longitude'] = JNorth_Larva_Site_2005_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [363]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_5_df, cols)

In [364]:
file = open("JNorth_Larva_Site_2005_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# June

In [365]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_6_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==6]
JNorth_Larva_Site_2005_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [366]:
JNorth_Larva_Site_2005_6_df.shape

(0, 6)

In [367]:
JNorth_Larva_Site_2005_6_df = pd.DataFrame(JNorth_Larva_Site_2005_6_df)
JNorth_Larva_Site_2005_6_df.drop(['month'], axis=1, inplace=True)

In [368]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_6_df['latitude'] = JNorth_Larva_Site_2005_6_df['latitude'].astype(float)
JNorth_Larva_Site_2005_6_df['longitude'] = JNorth_Larva_Site_2005_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [369]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_6_df, cols)

In [370]:
file = open("JNorth_Larva_Site_2005_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# July

In [371]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_7_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==7]
JNorth_Larva_Site_2005_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [372]:
JNorth_Larva_Site_2005_7_df.shape

(0, 6)

In [373]:
JNorth_Larva_Site_2005_7_df = pd.DataFrame(JNorth_Larva_Site_2005_7_df)
JNorth_Larva_Site_2005_7_df.drop(['month'], axis=1, inplace=True)

In [374]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_7_df['latitude'] = JNorth_Larva_Site_2005_7_df['latitude'].astype(float)
JNorth_Larva_Site_2005_7_df['longitude'] = JNorth_Larva_Site_2005_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [375]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_7_df, cols)

In [376]:
file = open("JNorth_Larva_Site_2005_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# August

In [377]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_8_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==8]
JNorth_Larva_Site_2005_8_df.tail()

sighting_date                species  latitude  longitude  number  month
0    2005-08-16  Monarch Larva Sighted    39.052    -77.125       5      8

In [378]:
JNorth_Larva_Site_2005_8_df.shape

(1, 6)

In [379]:
JNorth_Larva_Site_2005_8_df = pd.DataFrame(JNorth_Larva_Site_2005_8_df)
JNorth_Larva_Site_2005_8_df.drop(['month'], axis=1, inplace=True)

In [380]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_8_df['latitude'] = JNorth_Larva_Site_2005_8_df['latitude'].astype(float)
JNorth_Larva_Site_2005_8_df['longitude'] = JNorth_Larva_Site_2005_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [381]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_8_df, cols)

In [382]:
file = open("JNorth_Larva_Site_2005_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# September

In [383]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_9_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==9]
JNorth_Larva_Site_2005_9_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [384]:
JNorth_Larva_Site_2005_9_df.shape

(0, 6)

In [385]:
JNorth_Larva_Site_2005_9_df = pd.DataFrame(JNorth_Larva_Site_2005_9_df)
JNorth_Larva_Site_2005_9_df.drop(['month'], axis=1, inplace=True)

In [386]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_9_df['latitude'] = JNorth_Larva_Site_2005_9_df['latitude'].astype(float)
JNorth_Larva_Site_2005_9_df['longitude'] = JNorth_Larva_Site_2005_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [387]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_9_df, cols)

In [388]:
file = open("JNorth_Larva_Site_2005_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# October

In [389]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_10_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==10]
JNorth_Larva_Site_2005_10_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [390]:
JNorth_Larva_Site_2005_10_df.shape

(0, 6)

In [391]:
JNorth_Larva_Site_2005_10_df = pd.DataFrame(JNorth_Larva_Site_2005_10_df)
JNorth_Larva_Site_2005_10_df.drop(['month'], axis=1, inplace=True)

In [392]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_10_df['latitude'] = JNorth_Larva_Site_2005_10_df['latitude'].astype(float)
JNorth_Larva_Site_2005_10_df['longitude'] = JNorth_Larva_Site_2005_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [393]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_10_df, cols)

In [394]:
file = open("JNorth_Larva_Site_2005_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# November

In [395]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_11_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==11]
JNorth_Larva_Site_2005_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [396]:
JNorth_Larva_Site_2005_11_df.shape

(0, 6)

In [397]:
JNorth_Larva_Site_2005_11_df = pd.DataFrame(JNorth_Larva_Site_2005_11_df)
JNorth_Larva_Site_2005_11_df.drop(['month'], axis=1, inplace=True)

In [398]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_11_df['latitude'] = JNorth_Larva_Site_2005_11_df['latitude'].astype(float)
JNorth_Larva_Site_2005_11_df['longitude'] = JNorth_Larva_Site_2005_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [399]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_11_df, cols)

In [400]:
file = open("JNorth_Larva_Site_2005_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# December

In [401]:
JNorth_Larva_Site_2005_month_df =pd.DataFrame(JNorth_Larva_Site_2005_df)
JNorth_Larva_Site_2005_12_df = JNorth_Larva_Site_2005_month_df[JNorth_Larva_Site_2005_month_df['month']==12]
JNorth_Larva_Site_2005_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [402]:
JNorth_Larva_Site_2005_12_df.shape

(0, 6)

In [403]:
JNorth_Larva_Site_2005_12_df = pd.DataFrame(JNorth_Larva_Site_2005_12_df)
JNorth_Larva_Site_2005_12_df.drop(['month'], axis=1, inplace=True)

In [404]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Larva_Site_2005_12_df['latitude'] = JNorth_Larva_Site_2005_12_df['latitude'].astype(float)
JNorth_Larva_Site_2005_12_df['longitude'] = JNorth_Larva_Site_2005_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [405]:
geojson = df_to_geojson(JNorth_Larva_Site_2005_12_df, cols)

In [406]:
file = open("JNorth_Larva_Site_2005_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()